In [ ]:
%%bash
# The script to give header for filter file
for DATASET in $(ls ../dataset);do
    for FILE in $(ls ../dataset/$DATASET);do
        sed -i -e '1 s/Rsq/Rsq\n/' ../dataset/$DATASET/$FILE
    done
done

In [22]:
%%bash
# Create maf01rsq3
rm -f t/swarm_maf01rsq3.txt
for DATASET in $(ls dataset);do
    rm -f dataset/$DATASET/maf01rsq3.txt ##########################################################
    for i in {1..22};do
        echo "
zcat /data/LNG/CORNELIS_TEMP/progression_GWAS/$DATASET/chr"$i".info.gz | cut -f 1-4,7 |\
awk '\$4 > 0.01 && \$4 < 0.99 && \$5 > 0.3'{print} >> dataset/$DATASET/maf01rsq3.txt;\ " >> t/swarm_maf01rsq3.txt
    done
    echo "sed -i '1 iPOS\tA2\tA1\tALT_Frq\tRsq' dataset/$DATASET/maf01rsq3.txt" >> t/swarm_maf01rsq3.txt
done
echo "swarm -f t/swarm_maf01rsq3.txt -g 6 -b 5 --time=0:20:00 --logdir t/swarm --partition=quick --devel"


swarm -f t/swarm_maf01rsq3.txt -g 6 -b 100 --time=0:1:00 --logdir t/swarm --partition=quick --devel


In [73]:
%%bash
for DATASET in $(ls dataset/);do
    awk 'NR==1{print;next}{print|"LANG=C sort"}' dataset/$DATASET/maf01rsq3.txt > dataset/$DATASET/maf01rsq3_sorted.txt
done
head -n2 dataset/*/maf01rsq3_sorted.txt

==> dataset/CORIELL/maf01rsq3_sorted.txt <==
POS	A2	A1	ALT_Frq	Rsq
10:100000625	A	G	0.43157	0.79770

==> dataset/DATATOP/maf01rsq3_sorted.txt <==
POS	A2	A1	ALT_Frq	Rsq
10:100000625	A	G	0.42640	0.99974

==> dataset/DIGPD_chip/maf01rsq3_sorted.txt <==
POS	A2	A1	ALT_Frq	Rsq
10:100000625	A	G	0.41204	0.99958

==> dataset/DIGPD_neuroX/maf01rsq3_sorted.txt <==
POS	A2	A1	ALT_Frq	Rsq
10:100000625	A	G	0.39910	0.77992

==> dataset/HBS/maf01rsq3_sorted.txt <==
POS	A2	A1	ALT_Frq	Rsq
10:100000625	A	G	0.41867	0.82441

==> dataset/OSLO/maf01rsq3_sorted.txt <==
POS	A2	A1	ALT_Frq	Rsq
10:100000625	A	G	0.44188	0.99979

==> dataset/PARKFIT/maf01rsq3_sorted.txt <==
POS	A2	A1	ALT_Frq	Rsq
10:100000625	A	G	0.44278	0.99975

==> dataset/PARKWEST/maf01rsq3_sorted.txt <==
POS	A2	A1	ALT_Frq	Rsq
10:100000625	A	G	0.45917	0.99982

==> dataset/PDBP/maf01rsq3_sorted.txt <==
POS	A2	A1	ALT_Frq	Rsq
10:100000625	A	G	0.42939	0.81898

==> dataset/PICNICS/maf01rsq3_sorted.txt <==
POS	A2	A1	ALT_Frq	Rsq
10:100000625	A	G	0.41453	

In [74]:
%%bash
wc -l dataset/*/maf01rsq3.txt

   2815232 dataset/CORIELL/maf01rsq3.txt
   7756463 dataset/DATATOP/maf01rsq3.txt
   7659881 dataset/DIGPD_chip/maf01rsq3.txt
   2570686 dataset/DIGPD_neuroX/maf01rsq3.txt
   2965985 dataset/HBS/maf01rsq3.txt
   7733436 dataset/OSLO/maf01rsq3.txt
   7750642 dataset/PARKFIT/maf01rsq3.txt
   7823799 dataset/PARKWEST/maf01rsq3.txt
   2911368 dataset/PDBP/maf01rsq3.txt
   7688435 dataset/PICNICS/maf01rsq3.txt
   2783695 dataset/PPMI/maf01rsq3.txt
   2781980 dataset/PRECEPT/maf01rsq3.txt
   7767316 dataset/SCOPA/maf01rsq3.txt
  71008918 total


In [9]:
%%bash
# create maf01rsq3 info file (Required for rvstest)
for DATASET in $(ls dataset/);do
    cut -f1 dataset/$DATASET/maf01rsq3.txt | sed 's/:/\t/' | awk '{print $1":"$2"-"$2}' | sed '1d' > dataset/$DATASET/maf01rsq3.info
done

In [79]:
%%bash
# Get indiividual dosage of 20 K SNPs per file 
#### Time consuming. Creating swarm file takes 5 min
INPUT="/data/LNG/CORNELIS_TEMP/progression_GWAS"
FILTER=maf01rsq3
rm  t/swarm_DoseConv*
for DATASET in $(ls dataset/) ;do
  mkdir -p dataset/$DATASET/"$FILTER"_20Kcut 
  for CHR in {1..22};do
        grep ^$CHR: dataset/$DATASET/maf01rsq3.txt | cut -f1  | sed 's/:/\t/' | awk 'BEGIN{OFS="\t"}{print $1,$2,$2}' > t/t/$DATASET.$CHR.txt
        N_SNP=$(cat t/t/$DATASET.$CHR.txt | wc -l)
        N_ITER=$(expr $(expr $N_SNP - 1) / 20000)  # number of iteration (couting up from 0)
        for i in $(seq 0 $N_ITER);do
            START=$(expr 20000 \* $i + 1)
            if [ $i == $N_ITER ];then
                STOP=$N_SNP
            else
                STOP=$(expr 20000 \* $(expr $i + 1))
            fi
            echo "
sed -n "$START","$STOP"p t/t/$DATASET.$CHR.txt > t/t/$DATASET.$CHR.$i.txt;\
bcftools view -R t/t/$DATASET.$CHR.$i.txt $INPUT/$DATASET/chr"$CHR".dose.vcf.gz | bgzip -c > t/t/$DATASET.$CHR.$i.vcf.gz;\
DosageConvertor --vcfDose t/t/$DATASET.$CHR.$i.vcf.gz --type mach --format 1 --prefix t/t/$DATASET.$CHR.$i.filter;\
cat t/t/$DATASET.$CHR.$i.filter.mach.info | cut -f 1,2,3 | sed 's/\t/_/g' |\
 sed '1 cFID\nIID\nDOSE' | paste -s | sed 's/ /\t/g'> t/t/$DATASET.$CHR.$i.header.txt;\
zless t/t/$DATASET.$CHR.$i.header.txt t/t/$DATASET.$CHR.$i.filter.mach.dose.gz | sed 's/->/\t/g' | cut -f2- |\
 gzip > dataset/$DATASET/"$FILTER"_20Kcut/cut"$CHR"."$i".txt.gz;\
echo \"$DATASET $CHR $i $START,$STOP\" >> t/swarm_DoseConv_log.txt" >> t/swarm_DoseConv.txt
        done
    done
done

echo "swarm -f t/swarm_DoseConv.txt -g 1 -p 2 -b 20 --time=0:04:00 --module samtools,dosageconvertor --logdir t/swarm --partition=quick --devel"

swarm -f t/swarm_DoseConv.txt -g 1 -p 2 -b 20 --time=0:04:00 --module samtools,dosageconvertor --logdir t/swarm --partition=quick --devel


In [58]:
%%bash
## Correct the txt.gz file 
rm t/swarm_temp.txt
for DATASET in $(ls dataset/);do
    for i in $(ls dataset/$DATASET/maf01rsq3_20Kcut/);do
        echo "
zcat dataset/$DATASET/maf01rsq3_20Kcut/$i | sed -e 's/->/\t/g' -e '1 s/ID/FID\tIID/' | cut -f2- |\
gzip > dataset/$DATASET/maf01rsq3_20Kcut/$i.conv;\
mv dataset/$DATASET/maf01rsq3_20Kcut/$i.conv dataset/$DATASET/maf01rsq3_20Kcut/$i" >> t/swarm_temp.txt
    done
done

In [118]:
%%bash
gzcat dataset/DATATOP/maf01rsq3_20Kcut/cut1.0.txt.gz | head | cut -f 1-11
gzcat dataset/CORIELL/maf01rsq3_20Kcut/cut1.2.txt.gz | head | cut -f 1-11
swarm -f t/swarm_temp.txt -g 1 -p 2 -b 200 --time=0:01:00 --logdir t/swarm --partition=quick --devel

IID	DOSE	1:693731_A_G	1:706368_A_G	1:715265_C_T	1:715367_A_G	1:717485_C_A	1:717587_G_A	1:720381_G_T	1:721290_G_C	1:722670_T_C
50799_50799	DOSE	0.028	1.332	0.008	0.008	0.008	0.006	0.008	0.008	0.110
50072_50072	DOSE	0.021	1.368	0.006	0.006	0.006	0.003	0.006	0.006	0.084
50487_50487	DOSE	0.871	0.954	0.061	0.061	0.061	0.006	0.061	0.059	0.029
50614_50614	DOSE	0.729	0.374	0.065	0.065	0.065	0.412	0.065	0.057	0.718
50584_50584	DOSE	0.039	1.389	0.006	0.006	0.006	0.008	0.006	0.006	0.200
50671_50671	DOSE	0.895	0.864	0.094	0.098	0.098	0.007	0.099	0.099	0.041
50231_50231	DOSE	0.020	1.046	0.006	0.006	0.006	0.003	0.006	0.006	0.707
50065_50065	DOSE	0.023	1.551	0.007	0.007	0.007	0.004	0.007	0.007	0.088
50104_50104	DOSE	0.019	1.638	0.005	0.005	0.005	0.003	0.005	0.005	0.055
IID	DOSE	1:32140733_C_T	1:32142332_T_G	1:32142354_G_T	1:32143077_T_C	1:32143138_G_A	1:32146065_A_G	1:32146425_C_T	1:32146764_C_T	1:32147610_A_G
62219_62219	DOSE	0.977	0.839	0.836	0.150	0.866	0.005	0.996	1.006	1.940
21238_21238	DOSE	0.9

In [2]:
%%bash
ls -la dataset/*/maf01rsq3_20Kcut/ | awk '$5 < 100000 {print}'

dataset/CORIELL/maf01rsq3_20Kcut/:
total 1402152
drwxrws--- 2 iwakih2 LNG    20480 Sep 19 12:08 .
drwxr-sr-x 7 iwakih2 LNG     4096 Sep 18 16:37 ..
-rw-r--r-- 1 iwakih2 LNG    45324 Sep 19 11:41 cut2.12.txt.gz

dataset/DATATOP/maf01rsq3_20Kcut/:
total 1825184
drwxr-s--- 2 iwakih2 LNG   16384 Sep 19 10:44 .
drwxr-sr-x 7 iwakih2 LNG    4096 Sep 18 16:37 ..
-rw-r--r-- 1 iwakih2 LNG   33364 Sep 19 10:36 cut15.11.txt.gz

dataset/DIGPD_chip/maf01rsq3_20Kcut/:
total 542752
drwxr-s--- 2 iwakih2 LNG   16384 Sep 19 11:53 .
drwxr-sr-x 7 iwakih2 LNG    4096 Sep 18 16:38 ..
-rw-r--r-- 1 iwakih2 LNG   68731 Sep 19 10:34 cut11.19.txt.gz

dataset/DIGPD_neuroX/maf01rsq3_20Kcut/:
total 896144
drwxr-s--- 2 iwakih2 LNG    8192 Sep 19 11:53 .
drwxr-sr-x 7 iwakih2 LNG    4096 Sep 18 16:38 ..

dataset/HBS/maf01rsq3_20Kcut/:
total 4347280
drwxr-s--- 2 iwakih2 LNG     8192 Sep 19 11:00 .
drwxr-sr-x 7 iwakih2 LNG     4096 Sep 18 16:38 ..

dataset/OSLO/maf01rsq3_20Kcut/:
total 3770144
drwxr-s--- 2 iwakih2 LNG   

In [30]:
%%bash
# copy maf001rsq3 but it is not comlete set of files............Also, maf001rsq3 doesn't seem right. Need to be checked if use.
for DATASET in $(ls dataset/);do
    mkdir -p dataset/$DATASET/maf001rsq3_20Kcut
    cp /data/LNG/Hirotaka/progGWAS/SNPfilter/maf001rsq3_20Kcut/$DATASET/* dataset/$DATASET/maf001rsq3_20Kcut
done

In [ ]:
%%bash
# Old version but worked.
FILTER=maf001rsq3
# Transpose SNPs like the following format for the analysis in R.
## ....format....
## ID         SNP1    SNP2  ...
## Sample1    1.23    0.11
## Sample2...

# First cut each chromosome in a easy-operable number of SNPs (20K)
## "SEP" in R code will define the number. 
## Count the number of filtered SNPs in each chromosome 
rm -f _sep1.txt
for DATASET in $(cat Portfolios.txt) ;do
  for CHRNUM in {1..22};do
    echo $DATASET $CHRNUM `cat $GNTYP_OUT/SNPfilter/$DATASET"_"$FILTER"_"chr$CHRNUM.txt | wc -l` >> _sep1.txt
  done
done

## Then define the location to cut
echo '
options(scipen=999)
num = read.table("_sep1.txt")
SEP = 20000
num$V4 = num$V3/SEP
num$V5 = floor(num$V4) # The number of separation for the chromosome
t = rep(NA, nrow(num)*100) # the max snp is 120*10K so 100 would be more than enough
x = 1
for (i in 1:nrow(num)){
  for (j in 0:num$V5[i]){
    if(j==0){START=1}else{START=SEP*j} #First 2 are "ID" and "DOSE" not SNPs
    if(j==num$V5[i]){END=num$V3[i]}else{END=SEP*(j+1)-1}
    t[x] = paste(num$V1[i], ".", num$V2[i], ".", j+1, ".", START, ",", END, "p", sep = "")
    x = x + 1
  }
}
KEYS = data.frame(SEPKEY = t[!is.na(t)])
write.table(KEYS, "_sep2.txt", row.names = F, quote = F, sep = "\t")
' > _sep2.R
Rscript --vanilla _sep2.R

## Cut the SNPs with above defined locations and will use it for transposing pipeline.
mkdir $GNTYP_OUT/SNPfilter/$FILTER"_"20Kcut
FOLDER=$GNTYP_OUT/SNPfilter/$FILTER"_"20Kcut
for DATASET in $(cat Portfolios.txt) ;do
  mkdir -p $FOLDER/$DATASET
  for i in $(cat _sep2.txt | grep $DATASET);do
    CHNUM=$(echo $i | cut -d '.' -f 2)
    ITER=$(echo $i | cut -d '.' -f 3)
    AUG=$(echo $i | cut -d '.' -f 4) # location to cut. Will use for "sed -n $AUG ...."
    echo "
    cd $GNTYP_OUT/SNPfilter; \
    sed -n $AUG $GNTYP_OUT/SNPfilter/$DATASET"_"$FILTER"_"chr$CHNUM.txt | sed 's/:/\t/g' | sed 's/-/\t/g' > $FOLDER/$DATASET/chr$CHNUM.$ITER.txt.conv; \
    cd $FOLDER/$DATASET; \
    bcftools view -R chr$CHNUM.$ITER.txt.conv $GNTYP_IN/$DATASET/chr$CHNUM.dose.vcf.gz | bgzip -c > chr$CHNUM.$ITER.vcf.gz; \
    DosageConvertor --vcfDose chr$CHNUM.$ITER.vcf.gz \
      --type mach \
      --format 1 --prefix chr$CHNUM.$ITER.filter;\
    zless chr$CHNUM.$ITER.filter.mach.info | cut -f 1,2,3 | sed 's/\t/_/g' | sed '1d'> chr$CHNUM.$ITER.variant_list.txt; \
    echo DOSE > dose.txt; \
    echo ID > ID.txt; \
    cat ID.txt dose.txt chr$CHNUM.$ITER.variant_list.txt > chr$CHNUM.$ITER.final_variant_list.txt; \
    cut -f1 chr$CHNUM.$ITER.final_variant_list.txt | paste -s | sed 's/ /\t/g' > chr$CHNUM.$ITER.final_variant_list_trans.txt; \
    gunzip chr$CHNUM.$ITER.filter.mach.dose.gz; \
    cat chr$CHNUM.$ITER.filter.mach.dose | sed 's/->/\t/g' | cut -f2- > chr$CHNUM.$ITER.filter.mach.dose.format; \
    cat chr$CHNUM.$ITER.final_variant_list_trans.txt chr$CHNUM.$ITER.filter.mach.dose.format | gzip > chr$CHNUM.$ITER.trans.txt.gz; \
    rm -f chr$CHNUM.$ITER.f* chr$CHNUM.$ITER.v* chr$CHNUM.$ITER.txt* dose.txt ID.txt
    " >> _vcf_transtext.swarm
  done
done

# 5K subjobs so use 
rm -rf swarm_vcf_transtext
swarm -f _vcf_transtext.swarm -g 1 -p 2 -b 100 --time=0:04:00 --module samtools,dosageconvertor --logdir ./swarm_vcf_transtext


In [28]:
%%bash
module load bcftools
cut -f1 dataset/CORIELL/maf01rsq3.txt | head | sed 's/:/\t/g' | awk 'BEGIN{OFS="\t"}{print $1,$2,$2}' |\
bcftools view -R - $GNTYP_IN/$DATASET/chr$CHNUM.dose.vcf.gz | bgzip -c > chr$CHNUM.$ITER.vcf.gz;

POS		
1	829743	829743
1	836684	836684
1	864755	864755
1	864757	864757
1	876499	876499
1	879317	879317
1	879482	879482
1	879676	879676
1	879687	879687


In [6]:
%%bash
# Create maf01rsq3
# for DATASET in $(ls ../dataset);
DATASET=CORIELL
zcat /data/LNG/CORNELIS_TEMP/progression_GWAS/$DATASET/chr22.info.gz | cut -f 1-3,5,7 |\
awk 'NR==1{print;next}$4 > 0.01 && $4 < 0.99 && $5 > 0.3'{print} | head

SNP	REF(0)	ALT(1)	MAF	Rsq
22:17029525	T	G	0.05811	0.58847
22:17030029	A	G	0.05810	0.58960
22:17030126	T	A	0.05845	0.58726
22:17030792	G	A	0.05769	0.59623
22:17031072	C	T	0.05802	0.59290
22:17031494	A	G	0.05673	0.60812
22:17056280	C	A	0.05161	0.91357
22:17057138	G	T	0.05596	0.98953
22:17058560	A	G	0.05534	0.96134


In [5]:
%%bash
# Script to get maf05rsq8.info
for DATASET in $(ls dataset/);do 
    cat /data/LNG/Hirotaka/progGWAS/SNPfilter/DATATOP_maf05rsq8_*.info > dataset/$DATASET/maf05rsq8.info
    echo $DATASET finish
done

bash: line 2: syntax error near unexpected token `cat'
bash: line 2: `    cat /data/LNG/Hirotaka/progGWAS/SNPfilter/DATATOP_maf05rsq8_*.info > dataset/$DATASET/maf05rsq.info'


In [ ]:
%%bash
# Command to pull maf05rsq8 integer genotypes from the original file and cut per 20K genotypes
# Take sometime.  Maybe better to make swarm file.
mkdir t
for DATASET in $(ls dataset/);do
    plink --bfile /data/LNG/CORNELIS_TEMP/progression_GWAS/$DATASET/plink_files_hard/$DATASET.HARDCALLS \
        --extract dataset/$DATASET/maf05rsq8.info --recodeA include-alt --out t/"$DATASET"_extract
    mkdir -p dataset/$DATASET/maf05rsq8_20Kcut
    N_COL=$(head -n1 t/"$DATASET"_extract.raw | sed 's/ /\n/g' | wc -l)
    N_SNP=$(expr $N_COL - 6) # first 6 columns (FID IID PAT MAT SEX PHENOTYPE)
    N_ITER=$(expr $(expr $N_SNP - 1) / 20000)  # number of iteration (couting up from 0)
    echo $DATASET N_COL_$N_COL N_SNP_$N_SNP N_ITER_$N_ITER
    for i in $(seq 0 $N_ITER);do
        echo $i
        START=$(expr 20000 \* $i + 7)
        if [ $i == $N_ITER ];then
            STOP=$N_COL
            cut -d" " -f2,"$START"-"$STOP" t/"$DATASET"_extract.raw | sed 's/ /\t/g' > dataset/$DATASET/maf05rsq8_20Kcut/cut"$i".txt
        else
            STOP=$(expr 20000 \* $(expr $i + 1) + 6)
            cut -d" " -f2,"$START"-"$STOP" t/"$DATASET"_extract.raw | sed 's/ /\t/g' > dataset/$DATASET/maf05rsq8_20Kcut/cut"$i".txt
        fi
    done
done

rm -rf t

In [1]:
%%bash
# Create PCs from original arrayed-data. 
##  set exclusion_region
mkdir -p t
echo '5 44000000 51500000 r1
6 25000000 33500000 r2
8 8000000 12000000 r3
11 45000000 57000000 r4' > t/exclusion_regions_hg19.txt

module load plink

## Create PC1-5
GNTYP_IN=/data/LNG/CORNELIS_TEMP/progression_GWAS
for DATASET in $(ls dataset) ;do
  plink --noweb --bfile $GNTYP_IN/pre_impute_data/$DATASET \
  --exclude range t/exclusion_regions_hg19.txt --maf 0.05 --hwe 0.00001 --geno 0.05 --indep 50 5 2 --out t/$DATASET"_"pruning; \
  plink --noweb --bfile $GNTYP_IN/pre_impute_data/$DATASET \
  --extract t/$DATASET"_"pruning.prune.in --make-bed --out t/$DATASET"_"pruned; \
  plink --noweb --bfile t/$DATASET"_"pruned --pca 10 header tabs --out t/$DATASET.pca
done

# FID, IID in PC5 files are different from those in imputed files in some datasets. Correct this.
# temporary make eigenvec2 because PARKFIT returns blank if direct re-writing...
for DATASET in $(ls dataset) ;do
  if echo "$DATASET" | grep -q '^HBS\|^OSLO\|^PDBP\|^PPMI';then
    mv t/$DATASET.pca.eigenvec dataset/$DATASET/pca10.txt
    mv t/$DATASET.pca.eigenval dataset/$DATASET/pca10val.txt
    continue
  fi
  cat t/$DATASET*.eigenvec | awk '{print $1"_"$1"\t"$2"_"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7}' > t/$DATASET.pca.eigenvec2
  mv t/$DATASET.pca.eigenvec2 dataset/$DATASET/pca10.txt
  mv t/$DATASET.pca.eigenval dataset/$DATASET/pca10val.txt
done

PLINK v1.90b4.4 64-bit (21 May 2017)           www.cog-genomics.org/plink/1.9/
(C) 2005-2017 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to t/CORIELL_pruning.log.
Options in effect:
  --bfile /data/LNG/CORNELIS_TEMP/progression_GWAS/pre_impute_data/CORIELL
  --exclude range t/exclusion_regions_hg19.txt
  --geno 0.05
  --hwe 0.00001
  --indep 50 5 2
  --maf 0.05
  --noweb
  --out t/CORIELL_pruning

Note: --noweb has no effect since no web check is implemented yet.
257653 MB RAM detected; reserving 128826 MB for main workspace.
154188 variants loaded from .bim file.
340 people (0 males, 0 females, 340 ambiguous) loaded from .fam.
Ambiguous sex IDs written to t/CORIELL_pruning.nosex .
--exclude range: 5290 variants excluded.
--exclude range: 148898 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 340 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%

[+] Loading plink  1.9.0-beta4.4  on cn3110 


In [7]:
%%bash
for DATASET in $(ls dataset) ;do
  if echo "$DATASET" | grep -q '^HBS\|^OSLO\|^PDBP\|^PPMI';then
    mv t/$DATASET.pca.eigenvec dataset/$DATASET/pca5.txt
    continue
  fi
  cat t/$DATASET*.eigenvec | awk '{print $1"_"$1"\t"$2"_"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7}' > t/$DATASET.pca.eigenvec2
  mv t/$DATASET.pca.eigenvec2 dataset/$DATASET/pca5.txt
done

mv: cannot stat ‘t/HBS.pca.eigenvec’: No such file or directory
mv: cannot stat ‘t/OSLO.pca.eigenvec’: No such file or directory
mv: cannot stat ‘t/PDBP.pca.eigenvec’: No such file or directory
mv: cannot stat ‘t/PPMI.pca.eigenvec’: No such file or directory


In [ ]:
### Create A2 (Ref alleles) tables for corresponding chromosome positions
for DATASET in $(ls ../dataset/);do
    FILE=$GNTYP/$DATASET/plink_files_hard/$DATASET.HARDCALLS
    awk '{print $2,$6}' $FILE.bim | LANG=C sort | sed '1 iPOS A2' | sed 's/ /\t/g' > $FILE.A2
done


In [1]:
vcf

NameError: name 'vcf' is not defined